In [1]:
from sklearn . preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn . model_selection import train_test_split
import pandas as pd 
import pickle
import torch 
import torch.nn as nn
from torchinfo import summary
import os

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("radheshyamkollipara/bank-customer-churn")

print("Path to dataset files:", path)

d:\Gen AI\project\bank_customer_churn\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\shahj\.cache\kagglehub\datasets\radheshyamkollipara\bank-customer-churn\versions\1


In [3]:
# Find CSV files in the directory
csv_files = []
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('.csv'):
            csv_files.append(os.path.join(root, file))

print("Found CSV files:", csv_files)

# Load the first CSV file found
if csv_files:
    data = pd.read_csv(csv_files[0])
    print("Data loaded successfully!")
    print(data.head())
else:
    print("No CSV files found!")

Found CSV files: ['C:\\Users\\shahj\\.cache\\kagglehub\\datasets\\radheshyamkollipara\\bank-customer-churn\\versions\\1\\Customer-Churn-Records.csv']
Data loaded successfully!
   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0       2       0.00              1          1               1   
1       1   83807.86              1          0               1   
2       8  159660.80              3          1               0   
3       1       0.00              2          0               0   
4       2  125510.82              1          1   

### Preprocess the dataframe

In [4]:
# eliminate the irrelevant columns
data=data.drop(['RowNumber','CustomerId','Surname'], axis=1)
data.head(2)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1,1,2,DIAMOND,464
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,3,DIAMOND,456


In [5]:
data.describe(include='all')

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned
count,10000.000000,10000,10000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000,10000.000000
unique,NaN,3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN
top,NaN,France,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DIAMOND,NaN
freq,NaN,5014,5457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2507,NaN
mean,650.528800,NaN,NaN,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203800,0.204400,3.013800,NaN,606.515100
std,96.653299,NaN,NaN,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402842,0.403283,1.405919,NaN,225.924839
min,350.000000,NaN,NaN,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000,0.000000,1.000000,NaN,119.000000
25%,584.000000,NaN,NaN,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000,0.000000,2.000000,NaN,410.000000
50%,652.000000,NaN,NaN,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000,0.000000,3.000000,NaN,605.000000
75%,718.000000,NaN,NaN,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000,0.000000,4.000000,NaN,801.000000


In [159]:
# ---------- Label Encoding ----------
label_encoder = LabelEncoder()
data['Gender'] = label_encoder.fit_transform(data['Gender'])

# Save LabelEncoder
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

# ---------- One Hot Encoding ----------
one_hot_encoded = OneHotEncoder()
encoded_data_geography = one_hot_encoded.fit_transform(data[['Geography']])

# Convert to DataFrame
encoded_data = pd.DataFrame(
    encoded_data_geography.toarray(),
    columns=one_hot_encoded.get_feature_names_out(["Geography"])
)

# Merge with original data
data = pd.concat([data.drop('Geography', axis=1), encoded_data], axis=1)

# Save OneHotEncoder
with open("one_hot_encoder.pkl", "wb") as f:
    pickle.dump(one_hot_encoded, f)

# ---------- Get Dummies ----------
# data = pd.get_dummies(data, columns=['Card Type']).astype(int)

one_hot_encoded_card = OneHotEncoder()
encoded_data_card_type = one_hot_encoded_card.fit_transform(data[['Card Type']])

# Convert to DataFrame
encoded_data = pd.DataFrame(
    encoded_data_card_type.toarray(),
    columns=one_hot_encoded_card.get_feature_names_out(["Card Type"])
)

# Merge with original data
data = pd.concat([data.drop('Card Type', axis=1), encoded_data], axis=1)

# Save final processed data
with open("processed_data_card.pkl", "wb") as f:
    pickle.dump(one_hot_encoded_card, f)


In [160]:
# label_encoder=LabelEncoder()
# data['Gender']=label_encoder.fit_transform(data['Gender'])

# one_hot_encoded=OneHotEncoder()
# encoded_data_geography=one_hot_encoded.fit_transform(data[['Geography']])
# encoded_data=pd.DataFrame(encoded_data_geography.toarray(),columns=one_hot_encoded.get_feature_names_out(["Geography"]))
# data=pd.concat([data.drop('Geography',axis=1),encoded_data],axis=1)

# data = pd.get_dummies(data, columns=['Card Type']).astype(int)
data.head(2)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Point Earned,Geography_France,Geography_Germany,Geography_Spain,Card Type_DIAMOND,Card Type_GOLD,Card Type_PLATINUM,Card Type_SILVER
0,619,0,42,2,0.00,1,1,1,101348.88,1,1,2,464,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,1,3,456,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [161]:
# Decide the data independent and dependent 
x=data.drop('Exited', axis=1)
y=data['Exited']

# split the data in train and test sets
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

# scale these feature 
scaler=StandardScaler()
x_train_scaled=scaler.fit_transform(x_train)
x_test_scaled=scaler.transform(x_test)
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler, file)

### ANN Implementation


In [162]:
#  create sequential model 

model=nn.Sequential(
    nn.Linear(x_train_scaled.shape[1],64),  # Fully connected layer: input size=19, output size=64
    nn.ReLU(),
    nn.Linear(64,32),  # HL_1
    nn.ReLU(),
    nn.Linear(32,1), #HL-2
    # nn.Softmax()
    
)
print(model)

Sequential(
  (0): Linear(in_features=19, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=32, bias=True)
  (3): ReLU()
  (4): Linear(in_features=32, out_features=1, bias=True)
)


In [163]:
summary(model)

Layer (type:depth-idx)                   Param #
Sequential                               --
├─Linear: 1-1                            1,280
├─ReLU: 1-2                              --
├─Linear: 1-3                            2,080
├─ReLU: 1-4                              --
├─Linear: 1-5                            33
Total params: 3,393
Trainable params: 3,393
Non-trainable params: 0

In [164]:
# early stopping

class EarlyStopping:
    def __init__(self, patience=5,min_delta=0):
    
        """
        patience: number of epochs to wait for improvement
        min_delta: minimum change in monitored value to count as improvement
        """
        self.patience=patience
        self.min_delta=min_delta
        self.counter=0
        self.best_loss=None
        self.early_stop=False
        
    def __call__(self,val_loss):
        if self.best_loss is None:
            self.best_loss=val_loss
        elif val_loss> self.best_loss- self.min_delta:
            self.counter+=1
            print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter>=self.patience:
                self.early_stop=True
        else:
            self.best_loss=val_loss
            self.counter=0

In [165]:
# convert traning data to float tensor
x_train_scaled=torch.tensor(x_train_scaled, dtype=torch.float32)
y_train=torch.tensor(y_train.values, dtype=torch.float32).view(-1,1)

#Intialize early stopping
early_stopping=EarlyStopping(patience=10, min_delta=0.005)

# compile the model
optimizer=torch.optim.Adam(model.parameters(),lr=0.01)
criterion= nn.BCEWithLogitsLoss()

# tranin loop
for epoch in range(100):
    optimizer.zero_grad()
    y_pred=model(x_train_scaled)  # forward pass
    loss=criterion(y_pred,y_train)
    loss.backward() # Backpropgation
    optimizer.step()
    # print(f"Epoch {epoch+1}, Loss: {loss.item():4f}")
    
    # --- VALIDATION ---
    model.eval()
    with torch.no_grad():
        val_pred=model(x_train_scaled)
        val_loss=criterion(val_pred,y_train)
    print(f"Epoch {epoch+1}, Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")
    
    # --- Check EARLY STOP ---
    early_stopping(val_loss.item())
    if early_stopping.early_stop:
        print("Early stopping triggered!")
        break

Epoch 1, Train Loss: 0.7476, Val Loss: 0.6531
Epoch 2, Train Loss: 0.6531, Val Loss: 0.5722
Epoch 3, Train Loss: 0.5722, Val Loss: 0.5017
Epoch 4, Train Loss: 0.5017, Val Loss: 0.4473
Epoch 5, Train Loss: 0.4473, Val Loss: 0.4106
Epoch 6, Train Loss: 0.4106, Val Loss: 0.3821
Epoch 7, Train Loss: 0.3821, Val Loss: 0.3504
Epoch 8, Train Loss: 0.3504, Val Loss: 0.3125
Epoch 9, Train Loss: 0.3125, Val Loss: 0.2726
Epoch 10, Train Loss: 0.2726, Val Loss: 0.2358
Epoch 11, Train Loss: 0.2358, Val Loss: 0.2043
Epoch 12, Train Loss: 0.2043, Val Loss: 0.1772
Epoch 13, Train Loss: 0.1772, Val Loss: 0.1526
Epoch 14, Train Loss: 0.1526, Val Loss: 0.1289
Epoch 15, Train Loss: 0.1289, Val Loss: 0.1058
Epoch 16, Train Loss: 0.1058, Val Loss: 0.0841
Epoch 17, Train Loss: 0.0841, Val Loss: 0.0652
Epoch 18, Train Loss: 0.0652, Val Loss: 0.0497
Epoch 19, Train Loss: 0.0497, Val Loss: 0.0378
Epoch 20, Train Loss: 0.0378, Val Loss: 0.0289
Epoch 21, Train Loss: 0.0289, Val Loss: 0.0224
Epoch 22, Train Loss: 

In [166]:
# torch.save(model, 'model.pth')
torch.save(model, 'model.h5')

In [167]:
loaded_model = torch.load("model.h5", weights_only=False)
loaded_model.eval()

Sequential(
  (0): Linear(in_features=19, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=32, bias=True)
  (3): ReLU()
  (4): Linear(in_features=32, out_features=1, bias=True)
)

In [168]:
with open('label_encoder.pkl', 'rb') as file:
    encoder_gender= pickle.load(file)
    
with open('one_hot_encoder.pkl', 'rb') as file:
    encoder_geo= pickle.load(file)
    
with open('processed_data_card.pkl', 'rb') as file:
    encoder_card= pickle.load(file)
    
with open('scaler.pkl','rb') as file:
    scalers=pickle.load(file)



In [169]:
# example of input data
input_data={
    'CreditScore': 600,
    'Geography': 'France',
    'Age':40,
    'Gender':'Female',
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':5000,
    'Card Type':'DIAMOND',
    'Complain':2,
    'Satisfaction Score':2,
    'Point Earned':456
}
input_df = pd.DataFrame([input_data])

In [170]:
gender_encoded = encoder_gender.transform(input_df[['Gender']])
input_df = pd.concat([input_df.drop('Gender', axis=1).reset_index(drop=True), pd.DataFrame(gender_encoded, columns=['Gender'])], axis=1)

geo_encoder=encoder_geo.transform(input_df[['Geography']]).toarray()
input_df=pd.concat([input_df.drop('Geography',axis=1).reset_index(drop=True),pd.DataFrame(geo_encoder,columns=encoder_geo.get_feature_names_out(['Geography']))], axis=1)

card_encoder=encoder_card.transform(input_df[['Card Type']]).toarray()
input_df=pd.concat([input_df.drop('Card Type',axis=True).reset_index(drop=True),pd.DataFrame(card_encoder,columns=encoder_card.get_feature_names_out(['Card Type']))],axis=1)


d:\Gen AI\project\bank_customer_churn\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [171]:
input_df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Complain,Satisfaction Score,Point Earned,Gender,Geography_France,Geography_Germany,Geography_Spain,Card Type_DIAMOND,Card Type_GOLD,Card Type_PLATINUM,Card Type_SILVER
0,600,40,3,60000,2,1,1,5000,2,2,456,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [173]:
input_scaled=scalers.transform(input_df[scalers.feature_names_in_])

In [174]:
input_scaled

array([[-0.53598516, -1.09499335,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -1.65923237,  4.43195595,
        -0.72001005, -0.67025872,  1.00150113, -0.57946723, -0.57638802,
         1.72974448, -0.57715782, -0.58023704, -0.57388614]])

In [180]:
# predication
input_tensor=torch.tensor(input_scaled,dtype=torch.float32)

# Disable gradient calculation (for efficiency)
with torch.no_grad():
    output=loaded_model(input_tensor)
    
predication=torch.round(torch.sigmoid(output))
print("Prediction:", predication.numpy())


Prediction: [[1.]]
